## An example of crawling one page of Pantip.com

In [2]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re

### Assign some arguments

In [10]:
url = 'https://pantip.com/topic/38478051'

options = webdriver.ChromeOptions()
#options.add_argument('headless')

# initialize the driver
driver = webdriver.Chrome(executable_path="C:\webdriver\chromedriver.exe", options=options)
driver.get(url)

### Click button to accept cookies

![](cookies.png)


In [11]:
try:
    cookie = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.pt-snackbar__actions.pt-action-policy")))
    cookie.click()
except TimeoutException as ex:
    print("There is not the pop up")

### Click to load page

![](load_page.png)

In [12]:
loadPage = True
while loadPage:
    try:        
        load_comment_buttons = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.bar-paging-ed span.focus-txt")))
        driver.execute_script("return arguments[0].scrollIntoView({block: \"center\", inline: \"nearest\"});", load_comment_buttons)
        load_comment_buttons.click()
    except TimeoutException as ex:
        loadPage = False

### Click reply buttons

![](reply_button.png)

In [13]:
try:
    comment_buttons = WebDriverWait(driver, 3).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "a.reply.see-more span.focus-txt")))    
    for button in comment_buttons:
        driver.execute_script("return arguments[0].scrollIntoView({block: \"center\", inline: \"nearest\"});", button)
        button.click()
except TimeoutException as ex:
    print("No reply")

###  Loop click sub reply buttons
![](sub_reply_button.png)


In [14]:
loadButton = True   
while loadButton:
    try:
        load_comment_buttons = WebDriverWait(driver, 3).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "a.load-reply-next span.focus-txt"))) 
        for button in load_comment_buttons:
            driver.execute_script("return arguments[0].scrollIntoView({block: \"center\", inline: \"nearest\"});", button)
            button.click()
    except TimeoutException as ex:
        loadButton = False

### Get full HTML

In [15]:
content = driver.page_source
soup = BeautifulSoup(content,'lxml')
driver.close()

### Get the owner's post

In [16]:
pantip = dict()
all_comment = list()
pantip['title'] = str(soup.find(class_ = 'display-post-title').text).strip()
pantip['url'] =  url

main = {
        'post' : str(soup.find(class_ = 'display-post-story').text).strip(),
        'post_number' : 'ความคิดเห็นที่ 0',
        'owner' : str(soup.find(class_ = 'display-post-name owner').text).strip(),
        'time' : str(soup.find(class_ = 'timeago').text).strip()
        }
all_comment.append(main)
all_comment[0]

{'post': 'เพื่อนๆ คนไหนเคยทำ ช่วยแชร์ประสบการณ์หน่อยนะครับ',
 'post_number': 'ความคิดเห็นที่ 0',
 'owner': 'ทนายขีดเขียน',
 'time': '20 มกราคม เวลา 23:24 น.'}

### Loop get all posts

In [19]:
for div in soup.find_all("div", id = re.compile("^(comment|reply)-[0-9]+")):
    if len(str(div.text).strip()) > 1:
        post = str(div.find(class_ = 'display-post-story').text).strip()
        if post.find('ความคิดเห็นนี้ถูกลบเนื่องจาก') == -1:
            reply ={
                    'post' : post,
                    'post_number' : str(div.find(class_ = 'display-post-number').text).strip(),
                    'post_owner' : str(div.find(class_ = 'display-post-name').text).strip(),
                    'time' : str(div.find(class_ = 'timeago').text).strip()
                    }
            all_comment.append(reply)
pantip['sub_post'] = all_comment

### Data structure

![](data_structure.png)

In [23]:
pantip

{'title': 'อายุ 35 เงินเดือน 100,000 บาท อาศัยอยู่ใน กทม. เพื่อนๆ คิดว่า เพียงพอที่จะให้แฟนลาออกจากงานมาดูลูกอย่างเดียวไหมครับ',
 'url': 'https://pantip.com/topic/38478051',
 'sub_post': [{'post': 'เพื่อนๆ คนไหนเคยทำ ช่วยแชร์ประสบการณ์หน่อยนะครับ',
   'post_number': 'ความคิดเห็นที่ 0',
   'owner': 'ทนายขีดเขียน',
   'time': '20 มกราคม เวลา 23:24 น.'},
  {'post': 'ผญ.บางคนชอบทำงาน อยู่บ้านไม่มีสังคมจะเบื่อ ถามแฟนดีที่สุดครับ',
   'post_number': 'ความคิดเห็นที่ 1',
   'post_owner': 'สมาชิกหมายเลข 4054065',
   'time': '20 มกราคม เวลา 23:28 น.'},
  {'post': 'เงินเดือนขนาดนี้ ลาออกเลี้ยงลูกสบาย',
   'post_number': 'ความคิดเห็นที่ 2',
   'post_owner': 'สมาชิกหมายเลข 4158576',
   'time': '20 มกราคม เวลา 23:29 น.'},
  {'post': 'ถ้าปลอดหนี้ หรือมีหนี้ไม่มากทำได้สบาย\nมันอยู่ที่แฟนคุณเขาอยากลาออกหรือไม่มากกว่า',
   'post_number': 'ความคิดเห็นที่ 3',
   'post_owner': 'สมาชิกหมายเลข 3137301',
   'time': '20 มกราคม เวลา 23:42 น.'},
  {'post': 'แสนนึงไม่เยอะครับ ถ้าต้องผ่อนบ้านผ่อนรถ และยังไม่มีทรัพ